<a href="https://colab.research.google.com/github/FuriouStyles/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Stephen_P_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [124]:
#Borrowing my Haversine Distance function from my last assignment
from math import radians, cos, sin, asin, sqrt

def single_pt_haversine(lat, lng, degrees=True):
  r = 6371
  if degrees:
    lat, lng = map(radians, [lat, lng])
  a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
  d = 2*r*asin(sqrt(a))
  return d
df['harvesine_distance'] = [single_pt_haversine(LAT, LONG) for LAT, LONG in zip(df['latitude'], df['longitude'])]

#Create a new feature that contains the total number of amenities for that unit
amenities = pd.DataFrame({'1': df['balcony'],
              '2' : df['common_outdoor_space'],
              '3' : df['dishwasher'],
              '4' : df['exclusive'],
              '5' : df['fitness_center'],
              '6' : df['garden_patio'],
              '7' : df['hardwood_floors'],
              '8' : df['laundry_in_unit'],
              '9' : df['outdoor_space'],
              '10' : df['roof_deck'],
              '11' : df['swimming_pool'],
              '12' : df['terrace'],
              '13' : df['doorman'],
              '14' : df['high_speed_internet'],
              '15' : df['elevator'],
              '16' : df['new_construction']})

df['num_amenities'] = amenities.sum(axis=1)

# Categorically encoding of some features
df['display_address'] = df['display_address'].astype('category')
df['display_address_code'] = df['display_address'].cat.codes

df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_code'] = df['interest_level'].cat.codes

#Convert df.created from string to datatime object
df['created'] = pd.to_datetime(df['created'])
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,harvesine_distance,num_amenities,display_address_code,interest_level_code
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8661.842498,0,6418,2
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8665.491494,3,4404,1
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8667.251778,2,7245,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.737441,1,5590,1
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8664.656031,0,8125,1


In [125]:
type(df.created[0])

pandas._libs.tslibs.timestamps.Timestamp

### Fit a Linear Regression Model with Two New Features

In [0]:
#First we have to split our data
from datetime import datetime


date1 = datetime.strptime('2016-04-01', '%Y-%m-%d')
date2 = datetime.strptime('2016-05-31', '%Y-%m-%d')
date3 = datetime.strptime('2016-06-01', '%Y-%m-%d')


train = df.loc[(df['created'] >= date1) & (df['created'] <= date2)]
test = df.loc[df['created'] >= date3]

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

#Isolate our predictive (haversine distance and num of amenities) features
features = ['harvesine_distance',
            'bathrooms',
            'bedrooms',
            'dogs_allowed',
            'hardwood_floors',
            'laundry_in_unit',
            'display_address_code',
            'interest_level_code',
            'elevator',
            'doorman',
            'no_fee',
            'new_construction']
X_train = train[features]
X_test = test[features]

# Isolate our target (price) features
target = 'price'
y_train = train[target]
y_test = test[target]

# Scale our data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the model and make predictions - first our training data then our test data
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred_train = model.predict(X_train_scaled)
mae_train = mean_absolute_error(y_train, y_pred_train)

y_pred_test = model.predict(X_test_scaled)
mae_test = mean_absolute_error(y_test, y_pred_test)

In [206]:
# Get the model intercept and coefficient
model.intercept_, model.coef_

(3575.7377430197853,
 array([379.02877337, 849.70710148, 523.99183826,  22.61082774,
        -79.58494518, 207.24110305, -41.87080302, -66.35358213,
         65.95133224, 249.44993724, -83.34087142, -48.57074368]))

In [207]:
# Get the R^2 and RMSE

from sklearn.metrics import r2_score, mean_squared_error

mse = mean_squared_error(test[target], y_pred_test)
rmse = np.sqrt(mse)
r2 = r2_score(test[target], y_pred_test)
print(mse)
print(rmse)
print(mae_train)
print(r2)

1184728.0063580205
1088.452114866805
702.9795788758945
0.6188159911863829
